In [2]:
import numpy as np
import glob
import time
from model_robust import return_model
import torch

In [ ]:
def rgb2gray(rgb):

    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def get_fourier(imag):
    image_sample = np.fft.fft2(imag)
    F_orig = np.fft.fftshift(image_sample)
    F_orig = np.log(np.abs(F_orig))
    return F_orig

import time
import scipy.spatial
#attacks = ['PGDLinf', 'PGDL2', 'PGDL1', 'BBLinf', 'BBL2']
attacks = ['PGDLinf']
m = time.time()
namesss = ['fclinearl1relu', 'fclinearl3relu', 'convlinearl1k3c32relu', 'convlinearl3k3c32relu', 'lclinearl1k3c32relu', 'lclinearl3k3c32relu', 'lclinearl1k3c32reluconvinit', 'lclinearl3k3c32reluconvinit']

#namesss = ['lclinearl1k3c32']
for attack in attacks:
    for model in namesss:
        performance = 0
        total = 0
        checkpoint = torch.load('./cifar_resnet/ckpt_{}.pth'.format(model))
        maxs = checkpoint['epoch']
        print(model, maxs)
        fouriers = []
        fouriers_s = []
        for i in range(maxs,maxs+1):
            name = './cifar_resnet/{}_{}_adversarial.npy'.format(model,attack)
            name_s = './cifar_resnet/{}_saliency.npy'.format(model)
            b = np.load(name, allow_pickle=True)
            a = np.load(name_s, allow_pickle=True)
            b = b[0,:,:]
            #a = a[0,:,:]
            for j in range(b.shape[0]):
                if b[j,3] != None:
                    delta = b[j,0] - b[j,1]
                    delta_s = a[j]
                    delta = rgb2gray(delta)
                    fourier_s = np.abs(delta_s).mean(axis=0)
                    fourier_s = get_fourier(fourier_s)
                    fourier = get_fourier(delta)
                    fouriers.append(fourier)
                    fouriers_s.append(fourier_s)
                    performance = performance + 1
                total = total + 1

        fouriers = np.array(fouriers)
        fourier_s = np.array(fourier_s)
        np.save('./cifar_resnet/cifar_dynamics_adv_{}_fourier_{}.npy'.format(attack,model), fouriers)
        np.save('./cifar_resnet/cifar_dynamics_saliency_fourier_{}.npy'.format(model), fouriers_s)
        print('Model {}, Attack: {}, Adversarial Success: {}'.format(model, attack, performance/total))

